# SVM: Scikit-learn sobre dataset Labeled Faces in the Wild

## O que vamos fazer?
- Descarregar e analisar o dataset Labeled Faces in the Wild (LFW). 
- Pré-processar o dataset.
- Formar um modelo de classificação multiclasse com SVM por validação cruzada. 
- Avaliar a precisão do modelo e representá-la graficamente

O dataset Labeled Faces in the Wild (LFW) é um dos datasets mais usados como exemplo e para avaliar os nossos modelos e algoritmos. É um dataset muito curioso: é uma coleção de fotos de rostos de famosos (em formato JPEG) com 13 233 exemplos, 5 749 classes (rostos de famosos) e 5 828 características (pixeis).

Toda a informação está na sua página de internet oficial: http://vis-www.cs.umass.edu/lfw/

Habitualmente usamos o dataset em 2 tipos de tarefas ou modelos diferentes de classificação:
- Verificação de faces: num conjunto de 2 fotografias, um modelo de classificação binária deve prever se é a mesma pessoa.
- Reconhecimento facial: numa fotografia, o modelo deve classificá-la como pertencendo a um dos famosos identificados (classes).


A sua tarefa será criar um modelo SVM para resolver a tarefa de reconhecimento facial, otimizar os seus hiper-parâmetros por validação cruzada e avaliá-lo.

*Vamos formar um modelo ML que seja capaz de reconhecer rostos de famosos!*

Referências:
- [The Labeled Faces in the Wild face recognition dataset](https://scikit-learn.org/stable/datasets/index.html#labeled-faces-in-the-wild-dataset)
- [sklearn.datasets.fetch_lfw_people](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html)
- [sklearn.datasets.fetch_lfw_pairs](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_pairs.html)
- [Faces recognition example using eigenfaces and SVMs](https://scikit-learn.org/stable/auto_examples/applications/plot_face_recognition.html)

In [7]:
# TODO: Importar de todos os módulos necessários para esta célula

from sklearn.datasets import fetch_lfw_people
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from time import time
from scipy.stats import loguniform
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

## Carregar o dataset LFW

Representar graficamente alguns dos exemplos e as suas classes ou números associados.

*Nota*: Cuidado com o seu tamanho! Se for um dataset demasiado grande para o seu ambiente, pode reduzir o seu tamanho escolhendo um número de exemplos mais reduzido ao acaso, reordenando aleatoriamente os exemplos para evitar ter classes/faces com um número baixo de exemplos:

### Foi reduzido o Dataset, devido ao facto de ser muito extenso. Apenas faces com no minimo 20 exemplos foram escolhidas e cada imagem foi redimensionada para 30% do tamanho original 

In [2]:
# TODO: Carregar o dataset Digits como arrays X e Y e representar alguns dos exemplos
lfw_people = fetch_lfw_people(min_faces_per_person=20)#, resize=0.4)  # redimensiona as imagens para 30% do tamanho original

n_samples, h, w = lfw_people.images.shape

print(f"Exitem {n_samples} exemplos")
print(f"A imagem tem de tamanho {h} x {w}.")

Exitem 3023 exemplos
A imagem tem de tamanho 62 x 47.


In [3]:
X = lfw_people.data

# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names

print("Tamnho do dataset:", X.shape)
print("Número de classes:", len(target_names))
print("Nome das pessoas:", target_names)

Tamnho do dataset: (3023, 2914)
Número de classes: 62
Nome das pessoas: ['Alejandro Toledo' 'Alvaro Uribe' 'Amelie Mauresmo' 'Andre Agassi'
 'Angelina Jolie' 'Ariel Sharon' 'Arnold Schwarzenegger'
 'Atal Bihari Vajpayee' 'Bill Clinton' 'Carlos Menem' 'Colin Powell'
 'David Beckham' 'Donald Rumsfeld' 'George Robertson' 'George W Bush'
 'Gerhard Schroeder' 'Gloria Macapagal Arroyo' 'Gray Davis'
 'Guillermo Coria' 'Hamid Karzai' 'Hans Blix' 'Hugo Chavez' 'Igor Ivanov'
 'Jack Straw' 'Jacques Chirac' 'Jean Chretien' 'Jennifer Aniston'
 'Jennifer Capriati' 'Jennifer Lopez' 'Jeremy Greenstock' 'Jiang Zemin'
 'John Ashcroft' 'John Negroponte' 'Jose Maria Aznar'
 'Juan Carlos Ferrero' 'Junichiro Koizumi' 'Kofi Annan' 'Laura Bush'
 'Lindsay Davenport' 'Lleyton Hewitt' 'Luiz Inacio Lula da Silva'
 'Mahmoud Abbas' 'Megawati Sukarnoputri' 'Michael Bloomberg' 'Naomi Watts'
 'Nestor Kirchner' 'Paul Bremer' 'Pete Sampras' 'Recep Tayyip Erdogan'
 'Ricardo Lagos' 'Roh Moo-hyun' 'Rudolph Giuliani' 'Sadda

In [4]:
X[:5]

array([[0.22091503, 0.26013073, 0.47450984, ..., 0.07189543, 0.0627451 ,
        0.08366013],
       [0.26797387, 0.34509805, 0.26666668, ..., 0.03267974, 0.03137255,
        0.03006536],
       [0.07189543, 0.06013072, 0.05228758, ..., 0.06535948, 0.09019608,
        0.1006536 ],
       [0.3137255 , 0.5882353 , 0.7908497 , ..., 0.54509807, 0.5568628 ,
        0.5620915 ],
       [0.32287583, 0.29673204, 0.34509805, ..., 0.351634  , 0.3372549 ,
        0.29150328]], dtype=float32)

## Pré-processar os dados

Pré-processar os dados com os métodos Scikit-learn:

- Reordená-los aleatoriamente. 
- Normalizar, se necessário.
- Dividi-los em subsets de formação e testes.

Nesta ocasião, mais uma vez, faremos a validação cruzada por K-fold.

In [5]:
# TODO: Reordenar os dados aleatoriamente

indices = np.arange(len(X))
np.random.shuffle(indices)

X = X[indices]
y = y[indices]
X[:5]

array([[0.05490196, 0.08496732, 0.1006536 , ..., 0.3006536 , 0.34901962,
        0.3503268 ],
       [0.54771245, 0.51372546, 0.51111114, ..., 0.05490196, 0.05751634,
        0.05751634],
       [0.53986925, 0.53594774, 0.7267974 , ..., 0.18039216, 0.13986929,
        0.24444444],
       [0.19477125, 0.24444444, 0.29281047, ..., 0.32287583, 0.41830066,
        0.2771242 ],
       [0.36078432, 0.33202615, 0.29542485, ..., 0.54771245, 0.68235296,
        0.8104575 ]], dtype=float32)

In [8]:
# TODO: Normalizar, se necessário

scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X

array([[0.05511811, 0.08530184, 0.10104987, ..., 0.3006536 , 0.34901962,
        0.3503268 ],
       [0.5498688 , 0.515748  , 0.5131234 , ..., 0.05490196, 0.05751634,
        0.05751634],
       [0.5419947 , 0.53805774, 0.7296588 , ..., 0.18039216, 0.13986929,
        0.24444444],
       ...,
       [0.12335958, 0.05511811, 0.02230971, ..., 0.38300654, 0.5045752 ,
        0.57908493],
       [0.1456693 , 0.24671917, 0.4238845 , ..., 0.30718955, 0.29673204,
        0.28235295],
       [0.18766405, 0.20341207, 0.23228346, ..., 0.16862746, 0.19346406,
        0.19738562]], dtype=float32)

In [9]:
# TODO: Dividi-los em subsets de formação e testes

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)

## Formar um modelo de classificação por SVM inicial

Para comprovar o funcionamento do nosso classificador SVC antes de o otimizar por validação cruzada, vamos formar um modelo inicial sobre o subset de formação e validá-lo sobre o subset de teste.

Recordar usar a função [decision_function_shape](https://scikit-learn.org/stable/modules/svm.html#multi-class-classification) para usar o esquema “um contra o resto” (“ovr”). 

Usar os valores por defeito de *C* e *gamma* para não influir sobre a sua regularização:

In [10]:
model = SVC(decision_function_shape='ovr')
t0 = time()
model.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))

done in 14.045s


## Avaliar o modelo no subset de teste
Neste caso, não vamos representar a matriz de confusão, pois com 5749 classes seria demasiado grande para o analisar num gráfico:

In [11]:
# TODO: Avaliar o modelo com o seu score () no subset de teste.
t1 = time()
y_test_pred = model.predict(X_test)
print("done in %0.3fs" % (time() - t1))

done in 5.012s


In [12]:
print("Accuracy:", accuracy_score(y_test, y_test_pred))

Accuracy: 0.4193121693121693


### Então concluimos que, provavelmente por usar os valores por defeito de C e gamma, não obteve um bom resultado de accuracy nomeadamente de 0.42.